In [ ]:
import cv2   # opencv
import numpy as np
cap = cv2.VideoCapture(0)
if not cap.isOpened():  #만약 열려있지 않다면
    print("웹캠을 열 수 없습니다.")
    exit()
while True:   # 무한 루프
    ret, frame = cap.read()
    if not ret:  # 못 읽어오면 나가라
        print("프레임을 가져 올 수 없습니다.")
        break
    flip_frame = cv2.flip(frame,1)   # 프레임을 반전시켜라!
    height, width, _ =frame.shape  # 가운데 찾기
#  높이, 폭           컬러 자리 필요 없음
    center_x, center_y = width // 2, height // 2  # 가운데 맞춰줌
    roi = flip_frame[center_y - 150:center_y +150,
      center_x - 150:center_x +150]   #roi 공간 만들기
    cv2.rectangle(flip_frame, (center_x - 150, center_y - 150),(center_x + 150, center_y + 150)
                  ,(0,0,255), 2) # 빨간색 렉탱글이 정가운데 찍힘
    cv2.imshow("Webcam", flip_frame)
    # 화면 갭쳐용 키캆 받기
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c' or 'C'):   # 키값이 c나 C 면 캡쳐
        gray_image = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)  #BGR2GRAY 로 바꿔줌 이진화 흑백으로 바꿔줌
        gray_image = np.flip(gray_image,1) # 반전 시켜서 넣어줌
        cv2.imwrite("gray_image.png", gray_image)  #png 로 캡쳐본 저장
        gaussian_blur = cv2.GaussianBlur(gray_image,(5,5),3)  #가우시안 블러 처리
        # 2진화
        _, otsu_thread = cv2.threshold(gaussian_blur,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        cv2.imshow("otsu_thread", otsu_thread)
        # 전처리 (모핑  - Morph)
        kernel = np.ones((5,5),np.uint8)  # 5x5 의 컬러를 만들고
        erosion = cv2.erode(otsu_thread,kernel,iterations=1)  # 많이 반복할 수록 굵어짐. 배경이 흰색 숫자가 object라 확장필요
        cv2.imshow("erosion",erosion)
        cv2.imwrite("digit_binary_image.png", erosion)  #저장
        #이미지 자르기
        img = cv2.imread("digit_binary_image.png",cv2.IMREAD_UNCHANGED)
        h,w = img.shape[:2]
        crop_size = 280   # 자르기 상황에 따라 사이즈 변경 가능
        cx, cy = w// 2, h // 2 # 센터 잡기 센터로 짜를거니깐
        half = crop_size // 2
        x1, x2 = cx - half, cx + half
        y1, y2 = cx - half, cx + half
        # 경계면 설정
        x1 = max(0,x1)
        y1 = max(0,y1)
        x2 = min(w,x2)
        y2 = min(h,y2)

        # 이미지 가져오기
        cropped_img = img[y1:y2,x1:x2]
        cv2.imshow("cropped_img", cropped_img)  #크롭된 이미지 가져오기!

        # 이미지 반전o
        reversed_img = cv2.bitwise_not(cropped_img)    # 반전
        cv2.imshow("reversed_img", reversed_img)
        cv2.imwrite("IMAGE_FOR_TEST5.png", reversed_img)  # 이걸 가지고 테스트
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()